# 3. 키워드 반복으로 검색 쿼리 가중치를 조정하는 시스템 구축 
**목표**: 질문에서 핵심 키워드(예: '소득세')를 추출하고, 이를 강조한 검색 쿼리로 더 정확한 문서를 찾아봅니다.
1. 환경 설정
2. 원본 질문으로 검색 (Before)
3. 핵심 키워드 추출 
4. 키워드 강조 쿼리 생성
5. 가중치 적용 쿼리로 검색 (After)
6. Before vs After 유사도 비교
7. 실제 RAG 답변 생성 비교

## Step 1: 환경 설정
이전 실습에서 만든 DB 재사용 

In [1]:
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_chroma import Chroma

load_dotenv() 

llm = ChatUpstage()
embedding = UpstageEmbeddings(model='solar-embedding-1-large') # 임베딩 모델 
database = Chroma(
    collection_name='chroma-tax', 
    persist_directory="./chroma", 
    embedding_function=embedding
)

## Step 2: 원본 질문으로 검색 (Before)
질문(query)를 벡터화하여 DB에서 검색하고 유사도 순으로 3개의 문서를 반환한다. 
유사도 점수는 0에 가까울수록 유사함 

In [2]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

print("=" * 60)
print("[Before] 원본 질문으로 검색")
print(f"질문: {query}")
print("=" * 60)

results_before = database.similarity_search_with_score(query, k=3) # 유사도 점수와 함께 검색 

for i, (doc, score) in enumerate(results_before, 1):
    print(f"\n[문서 {i}] 유사도 점수: {score:.4f}")
    print(f"내용 미리보기: {doc.page_content[:150]}...")
    print("-" * 60)

[Before] 원본 질문으로 검색
질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?

[문서 1] 유사도 점수: 1.0449
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------

[문서 2] 유사도 점수: 1.0451
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------

[문서 3] 유사도 점수: 1.0451
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------


## Step 3: 핵심 키워드 추출
LLM을 사용해 전문용어를 우선 추출하기 

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

keyword_prompt = ChatPromptTemplate.from_template("""
다음 질문에서 가장 중요한 핵심 키워드 1-2개만 추출하세요.
법률이나 세금 관련 전문용어를 우선으로 추출해주세요.

질문: {question}

핵심 키워드만 쉼표로 구분하여 출력하세요:
""")

keyword_chain = keyword_prompt | llm | StrOutputParser() # 체인 구성 : 프롬프트 -> LLM -> 문자열 파싱 
keywords = keyword_chain.invoke({"question": query}) # 질문에서 키워드 추출 
print(f"추출된 핵심 키워드: {keywords}")

추출된 핵심 키워드: 소득세, 연봉


## Step 4: 키워드 강조 쿼리 생성

In [4]:
weighted_query = f"{keywords} {keywords} {keywords} {query}" # 키워드를 3번 반복하여 가중치 부여 

print("=" * 60)
print("[가중치 적용된 쿼리]")
print(weighted_query)
print("=" * 60)

[가중치 적용된 쿼리]
소득세, 연봉 소득세, 연봉 소득세, 연봉 연봉 5천만원인 직장인의 소득세는 얼마인가요?


## Step 5: 가중치 적용 쿼리로 검색 (After)

In [5]:
print("=" * 60)
print("[After] 키워드 가중치 적용 검색")
print(f"쿼리: {weighted_query}")
print("=" * 60)

results_after = database.similarity_search_with_score(weighted_query, k=3) # 가중치 적용된 질문(쿼리)으로 검색

for i, (doc, score) in enumerate(results_after, 1):
    print(f"\n[문서 {i}] 유사도 점수: {score:.4f}")
    print(f"내용 미리보기: {doc.page_content[:150]}...")
    print("-" * 60)

[After] 키워드 가중치 적용 검색
쿼리: 소득세, 연봉 소득세, 연봉 소득세, 연봉 연봉 5천만원인 직장인의 소득세는 얼마인가요?

[문서 1] 유사도 점수: 1.0048
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------

[문서 2] 유사도 점수: 1.0049
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------

[문서 3] 유사도 점수: 1.0049
내용 미리보기: 제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020....
------------------------------------------------------------


## Step 6: Before vs After 유사도 비교

In [6]:
print("\n" + "=" * 60)
print("검색 결과 비교")
print("=" * 60)

print(f"\n원본 질문: {query}")
print(f"추출된 키워드: {keywords}")
print(f"가중 쿼리: {weighted_query}")

print("\n[Before] Top 1 유사도:", f"{results_before[0][1]:.4f}")
print("[After] Top 1 유사도:", f"{results_after[0][1]:.4f}")

if results_after[0][1] < results_before[0][1]:
    improvement = ((results_before[0][1] - results_after[0][1]) / results_before[0][1]) * 100
    print(f"\n {improvement:.2f}% 개선!!!")
else:
    print("\n 큰 차이 없음")


검색 결과 비교

원본 질문: 연봉 5천만원인 직장인의 소득세는 얼마인가요?
추출된 키워드: 소득세, 연봉
가중 쿼리: 소득세, 연봉 소득세, 연봉 소득세, 연봉 연봉 5천만원인 직장인의 소득세는 얼마인가요?

[Before] Top 1 유사도: 1.0449
[After] Top 1 유사도: 1.0048

✅ 3.84% 개선!


## Step 7: 실제 RAG 답변 생성 비교

In [7]:
from langchain.chains import RetrievalQA
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt}
)

print("=" * 60)
print("[원본 질문 답변]")
print("=" * 60)
answer_before = qa_chain.invoke({"query": query})
print(f"\n답변:\n{answer_before['result']}")

print("\n" + "=" * 60)
print("[키워드 가중치 적용 답변]")
print("=" * 60)
answer_after = qa_chain.invoke({"query": weighted_query})
print(f"\n답변:\n{answer_after['result']}")

[원본 질문 답변]

답변:
연봉 5,000만원의 직장인이라면, 종합소득 과세표준 1,400만원 초과 5,000만원 이하의 세율을 적용합니다. 계산식은 '84만원 + (1,400만원을 초과하는 금액의 15퍼센트)'이므로, 소득세는 '84만원 + (3,600만원 * 0.15) = 1,114만원'이 됩니다.

[키워드 가중치 적용 답변]

답변:
연봉 5천만원인 직장인의 소득세는 84만원입니다. 과세표준 1,400만원 초과 5,000만원 이하의 소득세율은 15%로, 1,400만원을 초과하는 금액에 대해 해당 세율을 적용합니다. 따라서 (5,000만원 - 1,400만원) * 0.15 + 84만원 = 84만원이 소득세로 책정됩니다.
